# 2D Section construction from several independent meshes or Sections

The goal is to build, from `EdgedCurve2D` meshes or invalid `Sections` a valid `Section` with all needed topological information as defined here: https://docs.geode-solutions.com/datamodel.
A `Section` is the 2D equivalent for 3D `BRep`.


## Import modules

You need to import OpenGeode and Geode-BackgroundMesh modules.

In [ ]:
# Fix to better handle import since Python 3.8 on Windows
import os, sys, platform
if sys.version_info >= (3,8,0) and platform.system() == "Windows":
    for path in [x.strip() for x in os.environ['PATH'].split(';') if x]:
        os.add_dll_directory(path)

import opengeode
from geode_backgroundmesh import bmesh_2d # only the "bmesh_2d" part of the package is needed

## Load input data and initialize BackgroundMesh

In this section, we will see how to initialize the BackgroundMesh. You need to define the domain area of interest and a resolution. The domain area should to cover all the input data you need to insert in the BackgroundMesh. The resolution stands for the size of data regionalization in the BackgroundMesh: there is no need to define very fine resolution.

In [ ]:
# Load a Section
section = opengeode.load_section("input_section.og_sctn")
# Define resolution
bmesh_resolution = 1
# Initialize the BackgroundMesh to the Section BoundingBox
bmesh = bmesh_2d.BackgroundMesh2D(input_section.bounding_box(), bmesh_resolution)

## Insert data in BackgroundMesh

In this section, we will see how to insert data in the BackgroundMesh. This example shows with input Section, but equivalent method exists for EdgedCurve2D meshes.

In [ ]:
# Define a BackgroundMeshBuilder
builder = bmesh_2d.BackgroundMeshBuilder2D(bmesh)
# Insert section
builder.insert_model(input_section)
# You can also insert several EdgedCurve2D meshes, they will be combined to previous inserted elements: 
uuid1 = builder.insert_edged_curve(mesh1) 
uuid2 = builder.insert_edged_curve(mesh2) 
uuid3 = builder.insert_edged_curve(mesh3) 
uuid4 = builder.insert_edged_curve(mesh4) 
... 


## Extract and save output Sections

In this section, we will see how to get Sections from the BackgroundMesh. There are several ways to extract: all the inserted information, or a subtract defined from a BoundingBox or a list of inserted lines defining a closed domain. The Section is build with Corners, Lines and Surfaces.

In [ ]:
# Extract all inserted data gathered into a Section
output_section, mappings = bmesh_2d.extract_section(bmesh)
# the second return value "mappings" is for advanced usage and can be discarded,
# it offers the mappings between input line uuids and output ones.
 
# Save section
opengeode.save_section("output_section.og_sctn")
# Extract all inserted data gathered inside a BoundingBox as a Section
bbox = opengeode.BoundingBox2D()
bbox.add_point(opengeode.Point2D([10, 10]))
bbox.add_point(opengeode.Point2D([14, 14]))
output_section2, _ = bmesh_2d.extract_section_with_domain_bounding_box(bmesh, bbox)
# Extract all inserted data gathered inside a list of inserted lines defining a closed domain as a Section
output_section3, _ = bmesh_2d.extract_section_with_domain_limits(bmesh, [uuid1, uuid2, uuid3, uuid4])
